没看懂markdown里要写啥

In [1]:
#加载数据包
library(data.table)
library(stringr)
library(lubridate)
#1导入数据集
r.posts <- fread("posts.csv", na.strings = "", fill = TRUE)  
r.replys <- fread("replys.csv", na.strings = "", fill = TRUE)
#2、对r.posts进行操作
# 2.1 删除postnums全为0的行，查看post_id数量
r.posts <- r.posts[postnums != 0]  # 假设postnums是数值型，全0即等于0
post_id_count <- length(unique(r.posts$post_id))
cat("删除后post_id数量：", post_id_count, "\n")

# 2.2 统计“螺纹钢吧”帖子数量，对比postnums
lwg_posts <- r.posts[guba_name == "螺纹钢吧"]
lwg_count <- nrow(lwg_posts)
lwg_postnums_sum <- sum(lwg_posts$postnums, na.rm = TRUE)
cat("螺纹钢吧帖子数量：", lwg_count, "\n")
cat("螺纹钢吧postnums总和：", lwg_postnums_sum, "\n")
cat("更大的是：", ifelse(lwg_count > lwg_postnums_sum, "帖子数量", "postnums总和"), "\n")

# 2.3 按create_time升序，提取每个吧最早帖子并导出
# 先转换create_time为时间格式（确保排序正确）
r.posts[, create_time := ymd_hms(create_time, tz = "UTC")]
# 按guba_name分组，取create_time最小的行
earl.posts <- r.posts[, .SD[which.min(create_time)], by = guba_name]
# 导出为csv
fwrite(earl.posts, "earl.posts.csv")

#3、对r.replys进行操作
# 3.1 按post_id和reply_time分组升序，提取最晚回复
# 转换reply_time为时间格式
r.replys[, reply_time := ymd_hms(reply_time, tz = "UTC")]
# 按post_id分组，取reply_time最大的行
late.replys <- r.replys[, .SD[which.max(reply_time)], by = post_id]
# 导出
fwrite(late.replys, "late.replys.csv")

# 3.2 删除reply_content中带<>的部分
r.replys[, reply_content := str_remove_all(reply_content, "<.*?>")]  # 匹配所有<>包裹的内容并删除

#4、合并字段，生成表in.data
# 按post_id内连接（仅保留匹配的条目）
int.data <- merge(r.posts, r.replys, by = "post_id", all = FALSE)

#5、对int.data的操作
# 5.1 提取create_date，转换create_time格式
# 提取年月日，生成create_date
int.data[, create_date := as.Date(create_time, format = "%Y-%m-%d")]
# 转换create_time为POSXct标准格式（兜底验证）
int.data[, create_time := as.POSIXct(create_time, format = "%Y-%m-%d %H:%M:%OS")]

# 5.2 生成时间序列表
# 提取create_date的最大/最小值
date_min <- min(int.data$create_date, na.rm = TRUE)
date_max <- max(int.data$create_date, na.rm = TRUE)
# 生成日期序列
date_seq <- seq.Date(date_min, date_max, by = "day")
# 提取所有唯一guba_name
guba_list <- unique(int.data$guba_name)
# 生成笛卡尔积（每个吧对应每个日期）
tim <- CJ(guba_name = guba_list, create_date = date_seq)

# 5.3 合并tim与int.data，填充NA为0
int.data.ts <- merge(tim, int.data, by = c("guba_name", "create_date"), all.x = TRUE)
# 填充readnum和replynum的NA为0
int.data.ts[, `:=`(readnum = ifelse(is.na(readnum), 0, readnum),replynum = ifelse(is.na(replynum), 0, replynum))]

#6统计分析
#6.1 提取replynum最多的10条帖子和10个吧
# 10条帖子（按replynum降序取前10）
mx.post <- r.posts[order(-replynum)][1:10, .(post_id, guba_name, readnum, replynum)]
# 10个吧（按replynum总和降序取前10）
mx.ba <- r.posts[, .(total_readnum = sum(readnum, na.rm = TRUE),  total_replynum = sum(replynum, na.rm = TRUE)),  by = guba_name][order(-total_replynum)][1:10]
# 导出
fwrite(mx.post, "mx.post.csv")
fwrite(mx.ba, "mx.ba.csv")

# 6.2 描述性统计
st.int <- r.posts[, .(
  replynum_max = max(replynum, na.rm = TRUE),
  replynum_min = min(replynum, na.rm = TRUE),
  replynum_mean = mean(replynum, na.rm = TRUE),
  replynum_median = median(replynum, na.rm = TRUE),
  replynum_var = var(replynum, na.rm = TRUE),
  readnum_max = max(readnum, na.rm = TRUE),
  readnum_min = min(readnum, na.rm = TRUE),
  readnum_mean = mean(readnum, na.rm = TRUE),
  readnum_median = median(readnum, na.rm = TRUE),
  readnum_var = var(readnum, na.rm = TRUE)
)]
# 导出
fwrite(st.int, "stst.int.csv")










Attaching package: 'lubridate'


The following objects are masked from 'package:data.table':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from 'package:base':

    date, intersect, setdiff, union




删除后post_id数量： 253092 
螺纹钢吧帖子数量： 23209 
螺纹钢吧postnums总和： 575977753 
更大的是： postnums总和 


Warning message:
" 1 failed to parse."
Warning message:
" 9 failed to parse."
